## Toorls :
 
 Tools are enterface that an agent ,chain or LLm can use to interface with the word.

 tools like:
 ARXIV
 Wikipedia



 

In [2]:
!pip install -r requrements.txt

  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached langchain_openai-0.3.33-py3-none-any.whl.metadata (2.4 kB)
  Using cached chromadb-1.0.20-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_huggingface-0.3.1-py3-none-any.whl.metadata (996 bytes)
  Using cached langchain_chroma-0.2.5-py3-none-any.whl.metadata (1.1 kB)
  Using cached streamlit-1.49.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached langchain_ollama-0.3.7-py3-none-any.whl.metadata (2.1 kB)
  Using cached langchain_groq-0.3.8-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
  Using cached langserve-0.3.1-py3-none-any.whl.metadata (40 kB)
  Using cached opentelemetry_exporter_otlp_proto

### TOOLS CReation:
#### Search ENgine with tools and agents:


In [3]:
## Arxiv--Research
## Tools Creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [5]:
# Use the inbuild tools of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [6]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
Arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(Arxiv.name)

arxiv


In [7]:
tools=[wiki,Arxiv]


In [12]:
##Custom Tool[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
os.environ["HF_token"]=os.getenv("HF_token")
os.environ["Groq_key"]=os.getenv("Groq_key")
os.environ["Langchain_api_key"]=os.getenv("Langchain_api_key")
os.environ["Langchai_project"]="Tool_agent_llm_project"
Embeddings=HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

In [26]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documet=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=400)
split_doc=documet.split_documents(docs)
vectore_store=FAISS.from_documents(split_doc,embedding=Embeddings)
retriver=vectore_store.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E35E473E90>, search_kwargs={})

In [27]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool=create_retriever_tool(retriver,"langsith-search","Search any information about langsmith")
retriver_tool.name

'langsith-search'

In [28]:
tools=[wiki,Arxiv,retriver_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'f:\\study material\\NLP_Rag _Projects\\Langchain_searchengin_with_tools_agent\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsith-search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E35E54C180>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vecto

#### Run All Tool with Agents and LLM
 ##### Tools,LLM ---AgentExecution

In [29]:
groq_api_key=os.getenv("Groq_key")
llm=ChatGroq(groq_api_key=groq_api_key,model="llama-3.1-8b-instant")


In [32]:
## Prompt template

from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
## Agents
from langchain.agents import create_openai_tools_agent,initialize_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [35]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent, tools=tools,verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [38]:
agent_executer.invoke({"input":"Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsith-search` with `{'query': 'langsmith'}`


Get started with LangSmith - Docs by LangChainDocs by LangChain home pagePythonSearch...⌘KLangSmithPlatform for LLM observability and evaluationOverviewQuickstartsTrace an applicationEvaluate an applicationTest promptsAPI & SDKsAPI referencePython SDKJS/TS SDKPricingPlansPricing FAQDocs by LangChain home pagePythonSearch...⌘KAsk AIGitHubForumForumSearch...NavigationGet started with LangSmithGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGet startedObservabilityEvaluationPrompt engineeringSelf-hostingAdministrationGitHubForumGet started with LangSmithCopy pageCopy pageLangSmith is a platform for building production-grade LLM applications. Monitor and evaluate your application, so you can ship quickly and with confidence.
LangSmith is framework agnostic — you can use it with or without LangChain’s open source frameworks langchain and langgraph.

Start tracin

{'input': 'Tell me about langsmith',
 'output': 'The LangSmith platform is a subset of the larger LangChain framework.'}